In [1]:
import spacy
from spacy.tokens import Doc, SpanGroup
from spacy.matcher import Matcher
from zipfile import ZipFile
from pathlib import Path
from tqdm import autonotebook as tqdm
from spacy.training import biluo_tags_to_spans
import iobes
import re
from itertools import combinations
from spacy import displacy
import numpy as np
import math

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
data_dir = Path("./data/teaching-dataset")
with (data_dir / "relation_classification_text_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]
with (data_dir / "relation_classification_references_train.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("references.txt") as f:
        labels = []
        for line in f.read().decode("utf-8").split("\n"):
            relations = []
            for relation in re.finditer(r"\(\((\d+),(\d+)\),\((\d+),(\d+)\)\)", line):
                relation = (
                    (int(relation.group(1)), int(relation.group(2))),
                    (int(relation.group(3)), int(relation.group(4))),
                )
                relations.append(relation)
            labels.append(relations)

with (data_dir / "relation_classification_text_test.zip").open("rb") as file:
    zip_file = ZipFile(file)
    with zip_file.open("input.txt") as f:
        test_sentences = [
            sentence.split("\n") for sentence in f.read().decode("utf-8").split("\n\n")
        ]

In [4]:
def parse_sentence(sentence):
    words = []
    tags = []
    for item in sentence:
        word, tag = item.split(" ")
        words.append(word)
        tags.append(tag)
    doc = Doc(nlp.vocab, words=words)
    doc = nlp(doc)
    tags = iobes.bio_to_bilou(tags)
    doc.ents = biluo_tags_to_spans(doc, tags)
    return doc

In [5]:
def token_in_between_events(token, outer1, outer2):
    return  token.i <= outer1.start and token.i >= outer2.end or \
            token.i <= outer2.start and token.i >= outer1.end

def inside_event(event, token):
    return event.start <= token.i and event.end >= token.i

def events_inside_subtree(verb, event1, event2):
    a = False
    b = False
    for sub in verb.subtree:
        if inside_event(event1, sub):
            a = True
    for sub in verb.subtree:
        if inside_event(event2, sub):
            b = True
    return a and b

def backwards(verb, doc):
    keyword = None
    backwards = False
    for child in verb.children:
        if child.dep_ in ["agent"]:
            backwards = True
            keyword = child
        if child.text in ["from"]:
            backwards = True
            keyword = child
    next_word = doc[verb.i+1]
    if next_word.dep_ in ["aux"]:
        backwards = True
        keyword = next_word
    return backwards, keyword

def get_next_event(doc, position):
    lowest_distance = float('inf')
    for event in doc.ents:
        if abs(event.start - position) <= lowest_distance and position > event.end or position < event.start:
            next_event = event
            lowest_distance = abs(event.start - position)
            return next_event

def handle_cause_of(predictions, doc):
    for token in doc:
        if token.text == "cause" and doc[token.i+1].text in ["of", "for"]:
            for event1, event2 in combinations(doc.ents, 2):
                if token_in_between_events(token, event1, event2):
                    predictions.append(((event1.start, event1.end), (event2.start, event2.end)))
            if len(predictions) == 0:
                effect = get_next_event(doc, token.i)
                cause = get_next_event(doc, effect.end)
                predictions.append(((cause.start, cause.end), (effect.start, effect.end)))
    return predictions

def handle_because(predictions, doc):
    for token in doc:
        if token.text == "because" or token.text == "due" or token.text == "common" and doc[token.i+1].text == "with":
            for event1, event2 in combinations(doc.ents, 2):
                if token_in_between_events(token, event1, event2):
                    predictions.append(((event2.start, event2.end), (event1.start, event1.end)))
            if len(predictions) == 0:
                effect = get_next_event(doc, token.i)
                cause = get_next_event(doc, effect.end)
                predictions.append(((cause.start, cause.end), (effect.start, effect.end)))
    return predictions


def predict_2(sentence):
    doc = parse_sentence(sentence)
    events = doc.ents
    predictions = []

    predictions = handle_cause_of(predictions, doc)
    predictions = handle_because(predictions, doc)

    # find verbs
    verb_pattern = [[{'POS': 'VERB'}]]
    verb_matcher = Matcher(nlp.vocab)
    verb_matcher.add("verbs", verb_pattern)
    matches = verb_matcher(doc)
    verbs = [(doc[start:end]) for _, start, end in matches]
    verbs = [doc[verb.start] for verb in verbs] # get tokens instead of spans

    # remove verbs inside events
    for verb in verbs:
        for event in events:
            if verb.i >= event.start and verb.i < event.end:
                verbs.remove(verb)

    for event1, event2 in combinations(doc.ents, 2):
        # entities are actually in order
        for verb in verbs:
            is_backwards, keyword = backwards(verb, doc)
            if is_backwards:
                if events_inside_subtree(verb, event1, event2) and token_in_between_events(keyword, event1, event2):
                    predictions.append(((event2.start, event2.end), (event1.start, event1.end)))
            else:
                if events_inside_subtree(verb, event1, event2) and token_in_between_events(verb, event1, event2):
                    predictions.append(((event1.start, event1.end), (event2.start, event2.end)))

    # if len(predictions) == 0:
        for event1, event2 in combinations(doc.ents, 2):
        # entities are actually in order
            for verb in verbs:
                is_backwards, keyword = backwards(verb, doc)
                if is_backwards:
                    if token_in_between_events(keyword, event1, event2):
                        predictions.append(((event2.start, event2.end), (event1.start, event1.end)))
                else:
                    if token_in_between_events(verb, event1, event2):
                        predictions.append(((event1.start, event1.end), (event2.start, event2.end)))

    if len(predictions) == 0:
        for event1, event2 in combinations(doc.ents, 2):
            predictions.append(((event1.start, event1.end), (event2.start, event2.end)))

    predictions = set(predictions)

    return predictions
    


# for idx in np.arange(71,80,1):
for idx in [78]:
    doc = parse_sentence(sentences[idx])
    pred = predict_2(sentences[idx])

    if set(labels[idx]) != set(pred):
        print(doc)
        print(idx)
        print("Ground truth:")
        for cause, effect in labels[idx]:
            print("\t{} -> {}".format(doc[cause[0]:cause[1]], doc[effect[0]:effect[1]]))
        print("Predictions:")
        for cause, effect in pred:
            print("\t{} -> {}".format(doc[cause[0]:cause[1]], doc[effect[0]:effect[1]]))
        print("\n")

displacy.render(doc, style="dep")

Overall , IA is an autonomous syndrome , linked to damage in the left hemisphere involving semantic memory disorders rather than a defect in motor control . 
78
Ground truth:
	IA -> damage in the left hemisphere
	damage in the left hemisphere -> semantic memory disorders
Predictions:
	damage in the left hemisphere -> semantic memory disorders
	IA -> damage in the left hemisphere
	IA -> semantic memory disorders




In [6]:
spacy.explain("AUX")

'auxiliary'

In [7]:
predictions = []
for sentence in tqdm.tqdm(sentences):
    predictions.append(predict_2(sentence))

  0%|          | 0/468 [00:00<?, ?it/s]

In [8]:
def evaluate(predictions, references, micro_avg=True):
    tp = []
    fp = []
    fn = []
    for prediction, reference in zip(predictions, references):
        tp.append(len(set(prediction) & set(reference)))
        fp.append(len(set(prediction) - set(reference)))
        fn.append(len(set(reference) - set(prediction)))
    if micro_avg:
        tp = [sum(tp)]
        fp = [sum(fp)]
        fn = [sum(fn)]
    precision = [0 if tp[i] == 0 else tp[i] / (tp[i] + fp[i]) for i in range(len(tp))]
    recall = [0 if tp[i] == 0 else tp[i] / (tp[i] + fn[i]) for i in range(len(tp))]
    f1 = [
        0
        if precision[i] * recall[i] == 0
        else 2 * precision[i] * recall[i] / (precision[i] + recall[i])
        for i in range(len(tp))
    ]
    precision = sum(precision) / len(precision)
    recall = sum(recall) / len(recall)
    f1 = sum(f1) / len(f1)
    return precision, recall, f1

micro_precision, micro_recall, micro_f1 = evaluate(predictions, labels, True)
macro_precision, macro_recall, macro_f1 = evaluate(predictions, labels, False)

print("Micro Precision: {:.2f}".format(micro_precision))
print("Micro Recall: {:.2f}".format(micro_recall))
print("Micro F1: {:.2f}".format(micro_f1))
print("Macro Precision: {:.2f}".format(macro_precision))
print("Macro Recall: {:.2f}".format(macro_recall))
print("Macro F1: {:.2f}".format(macro_f1))


Micro Precision: 0.51
Micro Recall: 0.77
Micro F1: 0.61
Macro Precision: 0.70
Macro Recall: 0.81
Macro F1: 0.74


In [31]:
test_predictions = []
for sentence in tqdm.tqdm(test_sentences):
    test_predictions.append(predict_2(sentence))

pred_string = ""
for pred in test_predictions:
    pred_string += str(pred).replace("{","").replace("}","") + "\n"
    
pred_string = pred_string.replace(" ","")

with open("predictions.txt", "w") as f:
    f.write(pred_string)

  0%|          | 0/98 [00:00<?, ?it/s]